In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

In [2]:
engine = create_engine('sqlite:///college.db', echo=True)

In [3]:
meta = MetaData()

In [4]:
students = Table(
   'students', meta, 
   Column('id', Integer, primary_key=True), 
   Column('name', String), 
   Column('lastname', String),
)
meta.create_all(engine)

2022-01-13 13:14:37,831 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-01-13 13:14:37,842 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:37,847 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-01-13 13:14:37,848 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:37,868 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2022-01-13 13:14:37,870 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:37,875 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("students")
2022-01-13 13:14:37,879 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:37,886 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2022-01-13 13:14:37,893 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:38,078 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
parcours = Table(
   'parcours', meta, 
   Column('id_parcours', Integer, primary_key=True), 
   Column('name', String), 
   Column('student_id', Integer, ForeignKey("students.id")),
)
meta.create_all(engine)

2022-01-13 13:14:42,989 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2022-01-13 13:14:42,993 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:14:43,094 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("parcours")
2022-01-13 13:14:43,094 INFO sqlalchemy.engine.base.Engine ()


In [6]:
values = [(1, "Jean", "Dubois"), (2, "Florent", "Desbiolles")]
values

[(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]

In [7]:
# Créer la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=students.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

2022-01-13 13:14:58,831 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-01-13 13:14:58,835 INFO sqlalchemy.engine.base.Engine INSERT OR REPLACE INTO students VALUES (?,?,?)
2022-01-13 13:14:58,838 INFO sqlalchemy.engine.base.Engine [(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]
2022-01-13 13:14:58,845 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
values_parcours = [(1, "Data engineering", 1),(2, "Data Science", 2) ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values_parcours[0]))
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            ins = ins.format(tablename=parcours.name, markers=markers)
            connection.execute(ins, values_parcours)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

2022-01-13 13:15:02,639 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-01-13 13:15:02,643 INFO sqlalchemy.engine.base.Engine INSERT OR REPLACE INTO parcours VALUES (?,?,?)
2022-01-13 13:15:02,646 INFO sqlalchemy.engine.base.Engine [(1, 'Data engineering', 1), (2, 'Data Science', 2)]
2022-01-13 13:15:02,653 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM students;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM parcours;")
    print(results.fetchall())

2022-01-13 13:15:08,014 INFO sqlalchemy.engine.base.Engine SELECT * FROM students;
2022-01-13 13:15:08,018 INFO sqlalchemy.engine.base.Engine ()
[(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]
2022-01-13 13:15:08,065 INFO sqlalchemy.engine.base.Engine SELECT * FROM parcours;
2022-01-13 13:15:08,069 INFO sqlalchemy.engine.base.Engine ()
[(1, 'Data engineering', 1), (2, 'Data Science', 2)]


In [10]:
sql = text('DROP TABLE IF EXISTS students;')
result = engine.execute(sql)

2022-01-13 13:15:13,084 INFO sqlalchemy.engine.base.Engine DROP TABLE IF EXISTS students;
2022-01-13 13:15:13,088 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 13:15:13,418 INFO sqlalchemy.engine.base.Engine COMMIT
